การควบคุมเหมาะที่สุดและการเรียนรูู้เสริมกำลัง -- ดร.วโรดม ตู้จินดา

# ภาคผนวก C รวมการใช้งานซอฟต์แวร์เสริม

ในภาคผนวกนี้จะรวมการใช้งานเครื่องมือซอฟต์แวร์เสริมบนไพทอน รวมถึงการปรับแต่งแก้ไขเพื่อให้ทำงานได้ตามวัตถุประสงค์ ซึ่งรายละเอียดอาจมีมากเกินกว่าที่จะใส่ในเนื้อหาหลักของหนังสือ 

แพ็กเกจที่ใช้มากในหนังสือนี้คือ Drake [1] เป็นเครื่องมือซอฟต์แวร์สำหรับหุ่นยนต์ที่พัฒนาโดย Russ Tedrake และทีมงาน 
สนับสนุนโดย Toyota Research Institute (TRI) ในช่วงเริ่มต้น API ใช้ภาษา C++ เป็นหลัก 
แต่ในปัจจุบันรองรับภาษาไพทอนโดยผ่านแพ็กเกจ pydrake ซึ่งโค้ดในหนังสือนี้มีการใช้งานในหลายส่วน 
เช่นการหาอนุพันธ์อัตโนมัติ และการจำลองโมเดล อย่างไรก็ตามความช่วยเหลือเมื่อเกิดปัญหาอาจค่อนข้างจำกัด 
เนื่องจากชุมชนของ Drake ยังมีขนาดเล็ก ดังนั้นเนื้อหาในภาคผนวกนี้จะเกี่ยวข้องกับ Drake ค่อนข้างมาก โดยอาจใช้แพ็กเกจเสริมอื่นช่วยเช่น SymPy 


## C.1 การหาอนุพันธ์อัตโนมัติ

ในการควบคุมเหมาะที่สุด พลวัตของโมเดลมักอยู่ในรูปสมการอนุพันธ์ ดังนั้นมักต้องการการคำนวณหาอนุพันธ์ในโปรแกรม 
ซึ่งสามารถใช้วิธีการดังต่อไปนี้

**1. วิธีหาคำตอบเชิงสัญลักษณ์ (symbolic method)**

ใช้แพ็กเกจเสริมที่สามารถหาอนุพันธ์จากกฎหรือสูตรเหมือนกับที่เราใช้หาคำตอบบนกระดาษ

* ข้อดี
    * มีความแม่นยำสูงเทียบเท่าทางทฤษฎี
    * เข้าใจได้ง่าย

* ข้อเสีย
    * ต้องการแพ็กเกจที่สนับสนุนการคำนวณเชิงสัญลักษณ์เช่น SymPy
    * เมื่อพจน์มีความซับซ้อนมาก การคำนวณจะซับซ้อนตาม ทำให้ใช้เวลานานในการหาคำตอบ และผลลัพธ์อาจยังติดในรูปที่สามารถทำให้ง่ายกว่าโดยมนุษย์

**2. วิธีการประมาณค่าเชิงเลข (numerical method)** 

ใช้การประมาณค่า เช่นผลต่างจำกัด 
(finite difference) $f'(x) \approx \underset{h \rightarrow 0}{lim}\frac{f(x+h)-f(x)}{h}$  
(หรือใช็ $f'(x) \approx \underset{h \rightarrow 0}{lim}\frac{f(x+0.5h)-f(x-0.5h)}{h}$ จะได้ความแม่นยำมากขึ้น)

* ข้อดี
    * เขียนโค้ดง่าย ไม่ต้องการแพ็กเกจเสริมสำหรับคำนวณอนุพันธ์
    * ทำงานเร็ว

* ข้อเสีย
    * เป็นเพียงการประมาณค่าดังนั้นจึงมีค่าผิดพลาด โดยความแม่นยำขึ้นกับขนาดของขั้น $h$
    * เมื่อเลีอก $h$ มีค่าน้อยมาก จะเกิดปัญหาเชิงเลข

**3. วิธีการหาอนุพันธ์อัตโนมัติ (automatic differentiation)**

เป็นวิธีการหาอนุพันธ์โดยใช้ลำดับของการคำนวณขั้นพื้นฐานเช่นการบวกลบคูณหาร และฟังก์ชันเช่นเลขชี้กำลัง (exponential) ตรีโกณ ลอการิทึม ผ่านใช้กฎลูกโซ่ (chain rule)

* ข้อดี
    * ใช้การคำนวณน้อย ทำให้ได้คำตอบเร็ว
    * มีความแม่นยำสูงเทียบเท่าผลลัพธ์ทางทฤษฎี

* ข้อเสีย
    * ต้องการแพ็กเกจเสริมเช่น Drake, JAX, PyTorch, tensorflow,   ฯลฯ หรือหากเขียนโปรแกรมเองจะต้องมีการจัดเก็บค่าของฟังก์ชันและอนุพันธ์ไปพร้อมกัน

โดยข้อดีของวิธีการ 3 ทำให้เป็นที่นิยมในการหาอนุพันธ์สำหรับระบบที่มีความซับซ้อนเช่นโครงข่ายประสาทเทียม การหาอนุพันธ์อัตโนมัติสามารถแบ่งย่อยเป็น 2 ประเภทหลักคือ

* การคำนวณไปข้างหน้า (forward) 
* การคำนวณผันกลับ (reverse)

การเลือกใช้แบบใดจึงเหมาะสมขึ้นกับจำนวนอินพุตและเอาต์พุตของฟังก์ชัน เช่นหากมีอินพุตจำนวนมากและเอาต์พุตเป็นสเกลาร์หรือจำนวนน้อย (เช่นโครงข่ายประสาทเทียม) 
ควรเลือกวิธีการคำนวณผันกลับ แต่สำหรับพลวัตในหนังสือนี้มิใช่ประเด็นสำคัญ 
ดังนั้นจะเลือกใช้วิธีการคำนวณไปข้างหน้า ซึ่ง Drake รองรับผ่านแพ็กเกจย่อย forwarddiff


In [1]:
import numpy as np
from pydrake import forwarddiff

**ตัวอย่าง C.1** ทดสอบกับฟังก์ชันพื้นฐาน $y = f_1(x) = xsin(x)$

In [2]:
def f1(x):
    return x*np.sin(x)

อนุพันธ์มีค่าเท่ากับ $f_1'(x) = xcos(x) + sin(x)$ เมื่อกำหนดให้ $x = \frac{\pi}{6}$  
เอาต์พุตของฟังก์ชันและอนุพันธ์มีค่าเป็นดังนี้

In [11]:
x1 = np.pi/6
y = f1(x1)
print("y = {}".format(y))

y = 0.26179938779914935


In [13]:
dy = forwarddiff.derivative(f1,x1)
print("dy = {}".format(dy))

dy = 0.9534498410585543


เมื่อเปรียบเทียบกับการคำนวณด้วยมือ จะพบว่าความแตกต่างเป็นศูนย์

In [16]:
dy_a = x1*np.cos(x1) + np.sin(x1)
print("dy (คำนวณด้วยมือ) = {}".format(dy_a))
print("ค่าแตกต่าง = {}".format(np.linalg.norm(dy - dy_a)))

dy (คำนวณด้วยมือ) = 0.9534498410585543
ค่าแตกต่าง = 0.0


เปรียบเทียบกับวิธีประมาณค่าโดยผลต่างจำกัดโดยใช้ขั้นเวลาน้อยเช่น $h = 10^{-6}$ จะพบว่ามีความแม่นยำน้อยกว่า 
(ซึ่งหากใช้ค่า $h$ น้อยเกินไป จะเกิดปัญหาเชิงเลขทำให้ค่าผิดพลาดเพิ่มขึ้น)

In [23]:
h = 1e-6
dy_fd = (f1(x1+0.5*h) - f1(x1-0.5*h))/h
print("dy (ประมาณค่าเชิงเลข) = {}".format(dy_fd))
print("ค่าแตกต่าง = {}".format(np.linalg.norm(dy_fd - dy_a)))

dy (ประมาณค่าเชิงเลข) = 0.9534498409680126
ค่าแตกต่าง = 9.054179628265047e-11


**ตัวอย่าง C.2** กรณีที่ฟังก์ชันมีอินพุตเป็นเวกเตอร์และเอาต์พุตเป็นสเกลาร์เช่น $f_2(x) = x_1^2 + 2x_2^2$ 

In [24]:
def f2(x):
    return x[0]**2 + 2*x[1]**2

อนุพันธ์จะเป็นเวกเตอร์
$$
f_2'(x) = 
\left[\begin{array}{c} 
\frac{\partial f_2(x)}{\partial x_1} \\
\frac{\partial f_2(x)}{\partial x_2} 
\end{array}\right] = 
\left[\begin{array}{c} 
2x_1 \\
4x_2
\end{array}\right] \tag{C.1}
$$

เรียกว่าเกรเดียนต์ จะใช้ forwarddiff.gradient สมมุติว่ากำหนด $x_2 = [1.0, -1.0]$ ได้อนุพันธ์เป็นดังนี้ (ลองเปรียบเทียบกับการคำนวณด้วยมือจะเห็นว่าเท่ากัน)

In [26]:
x2 = np.array([1.0,-1.0])
grad_fx2 = forwarddiff.gradient(f2,x2)
print(grad_fx2)

[ 2. -4.]


**ตัวอย่าง C.3** ระบบพลวัตที่เป็นเงื่อนไขบังคับของการควบคุมเหมาะที่สุดเช่น $x_{k+1} = f(x_k,u_k)$ 
หรือในกรณีทั่วไปคือ $y = f(x,u)$ อยู่ในรูปหลายอินพุตหลายเอาต์พุต เพื่อความง่ายพิจารณากรณีฟังก์ชันไม่เป็นเชิงเส้นที่ไม่มีอินพุต $u$

$$
\left[\begin{array}{c}
y_1 \\
y_2
\end{array}\right] =
\left[\begin{array}{c}
x_1^3  + 2x_2 \\
x_1cos(x_2)
\end{array}\right] \tag{C.2}
$$

อนุพันธ์อยู่ในรูปเมทริกซ์เรียกว่าจาโคเบียน
$$
\left[\begin{array}{cc}
\frac{\partial y_1}{\partial x_1} & \frac{\partial y_1}{\partial x_2} \\
\frac{\partial y_2}{\partial x_1} & \frac{\partial y_2}{\partial x_2} \\
\end{array}\right] = 
\left[\begin{array}{cc}
3x_1^2 & 2 \\
cos(x_2) & -x_1sin(x_2) \\
\end{array}\right] \tag{C.3}
$$
 

In [32]:
def f3(x):
    y0 = x[0]**3  + 2*x[1]
    y1 = x[0]*np.cos(x[1])
    return [y0,y1]

คำนวณอนุพันธ์ที่อินพุต $x = [1, \pi /2]$ โดยใช้ forwarddiff.jacobian 

In [33]:
x3 = np.array([1.0, np.pi/2])
y3 = forwarddiff.jacobian(f3,x3)
print(y3)

[[ 3.000000e+00  2.000000e+00]
 [ 6.123234e-17 -1.000000e+00]]


### C.1.1 แก้ไขโค้ดจาก forwarddiff.py

โด้ดจากแพ็กเกจ forwarddiff ของ Drake เขียนไว้สำหรับปัญหาพื้นฐาน ซึ่งหลายครั้งต้องการการปรับแต่งเพื่อให้ได้ผลตามต้องการ 
ยกตัวอย่างเช่นหากมีฟังก์ชันที่มีอินพุตเป็นสเกลาร์แต่คืนค่าเอาต์พุตเป็นเวกเตอร์ ในกรณีนี้จะไม่มีเมธอดรองรับ 

**ตัวอย่าง C.4** พิจารณาฟังก์ชัน f4() ดังนี้

In [3]:
def f4(u):
    y0 = np.sin(u)
    y1 = np.cos(u)
    return [y0, y1]

ไม่สามารถใช้ forwarddiff.derivative() เพราะเอาต์พุตเป็นเวกเตอร์

In [12]:
# executing this cell should result in error
u0 = 1.0
forwarddiff.derivative(f4,u0)

AttributeError: 'list' object has no attribute 'derivatives'

ไม่สามารถใช้ forwarddiff.gradient() เพราะอินพุตต้องเป็นเวกเตอร์

In [13]:
# executing this cell should result in error
forwarddiff.gradient(f4,u0)

AssertionError: x must be a vector

วิธีการจัดการกับปัญหานี้ เริ่มจากนำเข้าแพ็กเกจเสริมสำหรับจัดการกับรูปแบบข้อมูลของ forwarddiff 
ที่เรียกว่า AutoDiffXd

In [6]:
from pydrake.autodiffutils import AutoDiffXd, ExtractGradient, ExtractValue, InitializeAutoDiff

นำโค้ดของฟังก์ชัน derivative() จาก forwarddiff.py มาแก้ไข ตั้งชื่อใหม่เป็น 
vectorout_derivative() 

In [7]:
def vectorout_derivative(function, x):
    """ origimal comment from forwarddiff.derivative()
    Compute the derivative of the function evaluated at the scalar input x
    using Eigen's automatic differentiation.

    The function should be scalar-input and scalar-output.
    """
    x_ad = AutoDiffXd(value=x, size=1, offset=0)
    y_ad = function(x_ad)
    y_derivative = ExtractGradient(y_ad) # changed
    return y_derivative

เรียกใช้ฟังก์ชันนี้ได้เหมือนกับฟังก์ชันอื่นๆ ที่สร้างขึ้นเอง 

In [8]:
u0 = 1.0
vectorout_derivative(f4,u0)

array([[ 0.54030231],
       [-0.84147098]])

ตรวจสอบคำตอบกับการคำนวณบนกระดาษ $f_4'(u) = [cos(u), -sin(u)]$ จะเห็นว่าถูกต้อง

In [11]:
np.array([np.cos(u0),-np.sin(u0)]).reshape(2,1)

array([[ 0.54030231],
       [-0.84147098]])

### C.1.2 การจัดเก็บข้อมูลตามโครงสร้าง AutoDiffXd

AutoDiffXd คือโครงสร้างข้อมูลที่นิยามโดย Drake สำหรับการคำนวณ forwarddiff 
หลักการคือเมื่อเรียกวิธีหนึ่งของ forwarddiff จะคำนวณเอาต์พุตของฟังก์ชันและอนุพันธ์ไปพร้อมกันและเก็บคำตอบรวมอยู่ในรูปของ 
AutoDiffXd ข้อเสียคือโครงสร้างนี้นิยามภายใน Drake โดยที่แพ็กเกจมาตรฐานทั่วไปเช่น numpy ไม่รองรับ 
ดังนั้นจึงมักเกิดข้อผิดพลาดขึ้นเสมอเมื่อใช้ร่วมกับ numpy (นี้เป็นเหตุผลที่บางแพ็กเกจที่ใช้คำนวณอนุพันธ์อัตโนมัติเช่น JAX 
จะมาพร้อมกับ jax.numpy ของตนเอง)

ประเด็นที่เพิ่มความซับซ้อนคือโดยทั่วไปแล้วขนาดมิติของข้อมูลเอาต์พุตของฟังก์ชันและอนุพันธ์จะแตกต่างกัน ในกรณีของจาโคเบียน 
ค่าอนุพันธ์อาจอยู่ในรูปเทนเซอร์ เราลองมาทำความเข้าใจกับ AutoDiffXd จากฟังก์ชันเดิมในตัวอย่าง C.3

**ตัวอย่าง C.5** จากฟังก์ชัน f3(x) เดิม สร้างตัวแปรส่วนกลาง (global variable) เพื่อตรวจสอบโครงสร้าง AutoDiffXd

In [17]:
def f5(x):
    global x_global, y_global
    x_global = x
    y = [0,0]
    y[0] = x[0]**3  + 2*x[1]
    y[1] = x[0]*np.cos(x[1])
    y_global = y
    return y

In [18]:
x5 = np.array([1.0, np.pi/2])
y5 = forwarddiff.jacobian(f5,x5)

ตรวจสอบอินพุตจากตัวแปร x_global และ y_global จะเห็นว่าเป็นแบบ AutoDiffXd ที่มีมิติของ nderiv 
เพิ่มเข้ามา

In [19]:
x_global

array([<AutoDiffXd 1.0 nderiv=2>,
       <AutoDiffXd 1.5707963267948966 nderiv=2>], dtype=object)

In [21]:
y_global

[<AutoDiffXd 4.141592653589793 nderiv=2>,
 <AutoDiffXd 6.123233995736766e-17 nderiv=2>]

เราสามารถแยกเฉพาะข้อมูลเอาต์พุตของฟังก์ชันโดยคำสั่ง

In [22]:
ExtractValue(y_global)

array([[4.14159265e+00],
       [6.12323400e-17]])

หรือหากต้องการเฉพาะค่าอนุพันธ์ ใช้คำสั่ง

In [23]:
ExtractGradient(y_global)

array([[ 3.000000e+00,  2.000000e+00],
       [ 6.123234e-17, -1.000000e+00]])

ซึ่งก็คือคำตอบของ forwarddiff.jacobian() ในตัวอย่าง C.3 นั่นเอง เมื่อเข้าใจโครงสร้างภายในของ 
AutoDiffXd อาจช่วยให้เราหาหนทางแก้ปัญหาได้กรณีที่เกิดความขัดแย้งกันระหว่าง forwarddiff กับ numpy

**ตัวอย่าง C.6** พิจารณาฟังก์ชันที่รับอินพุต $x$ เป็นเวกเตอร์ 2 สมาชิก $u$ เป็นสเกลาร์ สร้างเมทริกซ์ 
$M$ ขนาด 2x2 จาก $x$ เวกเตอร์ $\tau$ จาก $u$ และคืนค่า y = np.linalg.solve($M,\tau$) 

In [29]:
from numpy.linalg import solve, inv

In [37]:
def f6(x,u):
    s1 = np.sin(x[0])
    c1 = np.cos(x[0])
    s12 = np.sin(x[0]+x[1])    
    c12 = np.cos(x[0]+x[1])
    m11 = s1 + c12
    m12 = c1 + s12
    m22 = 2.0
    M = np.array([[m11,m12],[ m12,m22]])
    tau = np.array([0,u]).reshape(2,1)
    y = solve(M,tau)
    return y

In [38]:
x6 = np.array([-np.pi/4, np.pi/2])
u6 = 1.0
f6(x6,u6)

array([[ 7.07106781e-01],
       [-5.55111512e-17]])

ต้องการหาอนุพันธ์ย่อยของฟังก์ชันนี้เทียบกับ $x$ และ $u$ 

ประเด็นแรกที่ต้องพิจารณาคือ forwarddiff ของ Drake ไม่ได้เขียนเพื่อรองรับฟังก์ชันที่มีอาร์กิวเมนต์หลายตัว 
วิธีการที่ผู้เขียนใช้คือสร้างตัวแปรส่วนกลางสำหรับอาร์กิวเมนต์ตัวที่ไม่ได้ใช้คำนวณอนุพันธ์ สร้างเป็นฟังก์ชันห่อหุ้มสำหรับสองกรณีดังนี้

In [34]:
def f6_x(x):
    global u_global
    return f6(x,u_global)

def f6_u(u):
    global x_global
    return f6(x_global, u)

เมื่อใช้ forwardiff.jacobian() พบว่าสามารถผ่านฟังก์ชันห่อหุ้มได้ แต่จะติดปัญหาตรงหาคำตอบ y

In [39]:
# running this cell should result in error
x6 = np.array([-np.pi/4, np.pi/2])
u6 = 1.0
u_global = u6
forwarddiff.jacobian(f6_x,x6)

UFuncTypeError: Cannot cast ufunc 'solve' input 0 from dtype('O') to dtype('float64') with casting rule 'same_kind'

เหตุผลคือฟังก์ชัน numpy.linalg.solve() ไม่รองรับตัวแปร AutoDiffXd ที่สร้างโดย forwarddiff 
การแก้ไขโค้ดตรงส่วนนี้ต้องอาศัยการคำนวณบนแผ่นกระดาษเข้าช่วย และเราหลีกเลี่ยงการใช้ solve() ของ numpy 
โดยใช้การหาคำตอบในรูป y = np.linalg.inv($M$)$\tau$ แทน ซึ่งจะแก้ได้ง่ายกว่า 

พิจารณาสมการเชิงเส้น

$$
My = \tau \tag{C.4}
$$
เมื่อหาอนุพันธ์ตลอดสมการและหาคำตอบ $\dot{y}$ 
$$
\begin{align*}
\dot{M}y + M\dot{y} &= \dot{\tau} \\
M\dot{y} &= \dot{\tau} - \dot{M}y \\
\dot{y} &= M^{-1}(\dot{\tau} - \dot{M}y) \tag{C.5}
\end{align*}
$$

สังเกตว่ากรณีอนุพันธ์เทียบ $x$ เวกเตอร์ $\tau$ เป็นค่าคงที่ ดังนั้น $\dot{\tau} = 0$ คำตอบจะเหลือเพียง
$$
\dot{y} = M^{-1}\dot{M}y \tag{C.6}
$$

    
ส่วนกรณีอนุพันธ์เทียบ $u$ เมทริกซ์ $M$ เป็นค่าคงที่ ดังนั้น $\dot{M} = 0$ คำตอบจะเหลือเพียง
$$
\dot{y} = M^{-1}\dot{\tau} \tag{C.7}
$$

เราจะเพิ่มตัวแปร adflag สำหรับจัดการคำตอบแต่ละกรณี โดยค่า 0,1,2 แทนค่าเอาต์พุตของฟังก์ชัน อนุพันธ์เทียบ $x$ และ $u$ 
ตามลำดับ เมื่อดัดแปลงแก้ไขแล้วจะได้เป็นฟังก์ชัน f6a() ดังนี้

In [82]:
def f6a(x,u):
    s1 = np.sin(x[0])
    c1 = np.cos(x[0])
    s12 = np.sin(x[0]+x[1])    
    c12 = np.cos(x[0]+x[1])
    m11 = s1 + c12
    m12 = c1 + s12
    m22 = 2.0
    M = np.array([[m11,m12],[ m12,m22]])
    tau = np.array([0,u]).reshape(2,1)
    if not adflag: # adflag: # not called from forwarddiff
        y = solve(M,tau)
    else:
        M_value = ExtractValue(M)
        tau_value = ExtractValue(tau)
        M_grad = ExtractGradient(M)
        tau_grad = ExtractGradient(tau)
        y_value = solve(M_value, tau_value)
        if adflag==1: # Jacobian w.r.t x
            dy_dx1 = inv(M_value)@(-M_grad[:,0].reshape(2,2)@y_value)
            dy_dx2 = inv(M_value)@(-M_grad[:,1].reshape(2,2)@y_value)
            y_grad = np.hstack((dy_dx1,dy_dx2))
            y = InitializeAutoDiff(y_value, y_grad)
        else: # derivative w.r.t u
            y_grad = inv(M_value)@ tau_grad
            y = InitializeAutoDiff(y_value,y_grad)
    return y

จุดที่ต้องระวังในกรณีหาอนุพันธ์เทียบ $x$ คือ $\dot{M}$ คืออนุพันธ์ย่อยของแต่ละสมาชิกของเมทริกซ์ $M$ 
เทียบกับแต่ละสมาชิกของเวกเตอร์ $x$ ดังนั้นจึงมีขนาด 4 x 2 จึงต้องจัดรูปให้ถูกต้องก่อนคำนวณ (C.6)

นิยามฟังก์ชันห่อหุ้มสำหรับ f6a() ดังนี้

In [41]:
def f6a_x(x):
    global u_global
    return f6a(x,u_global)

def f6a_u(u):
    global x_global
    return f6a(x_global, u)

นิยามฟังก์ชันหาอนุพันธ์เทียบ $x$ และ $u$ ชื่อว่า dfdx(), dfdu() ตามลำดับ โดยสำหรับ dfdu() 
จะใช้ฟังก์ชัน vectorout_derivative() ตามที่อธิบายในตัวอย่าง

In [ ]:
def vectorout_derivative(function, x):
    """ origimal comment from forwarddiff.derivative()
    Compute the derivative of the function evaluated at the scalar input x
    using Eigen's automatic differentiation.

    The function should be scalar-input and scalar-output.
    """
    x_ad = AutoDiffXd(value=x, size=1, offset=0)
    y_ad = function(x_ad)
    y_derivative = ExtractGradient(y_ad) # changed
    return y_derivative

In [ ]:
def dfdx(x,u):
    global adflag
    adflag = 1
    u_global = u
    return np.squeeze(forwarddiff.jacobian(f6a_x,x))

def dfdu(x,u):
    global adflag
    adflag = 2
    x_global = x
    return vectorout_derivative(f6a_u,u)
    

ทดสอบฟังก์ชัน f6a() สำหรับการคำนวณค่าเอาต์พุต การหาอนุพันธ์ย่อยเทียบ $x$ และ $u$  

In [83]:
x6 = np.array([-np.pi/4, np.pi/2])
u6 = 1.0
adflag = 0
f6a(x6,u6)

array([[ 7.07106781e-01],
       [-5.55111512e-17]])

In [84]:
dfdx(x6,u6) # derivative w.r.t x

array([[-7.07106781e-01, -8.53553391e-01],
       [ 5.55111512e-17,  3.53553391e-01]])

In [85]:
dfdu(x6,u6) # derivative w.r.t u

array([[0.92540786],
       [0.        ]])

### C.1.3 การประมาณค่าโมเดลเชิงเส้น

ในเนื้อหาที่ตามรอยรายวิชา [2] นิยมใช้การหาอนุพันธ์อัตโนมัติเพื่อประมาณค่าโมเดลเชิงเส้นแบบดีสครีตจากพลวัตไม่เป็นเชิงเส้น 
โดยอาศัยฟังก์ชันคำนวณอนุพันธ์โดยวิธีรุงเง คุตตา ในหัวข้อนี้จะทดสอบวิธีการดังกล่าวโดยเปรียบเทียบกับการประมาณค่าบนกระดาษ

**ตัวอย่าง C.7** พลวัตของคาร์ทโพล [3]

เชื่อว่าผู้ศึกษาระบบควบคุมคงเคยเห็นหรือคุ้นเคยกับคาร์ทโพลในรูปที่ 1 หรืออาจรู้จักในชื่อ "ลูกตุ้มหัวกลับบนรถเลื่อน" 
วัตถุประสงค์การควบคุมคือรักษาดุลให้ลูกตุ้มอยู่ในแนวดิ่ง เรียกว่าเป็นจุดสมดุลที่ไม่เสถียร (unstable equilibrium) 
โดยอาศัยแรงในแนวนอนที่กระทำกับรถเข็นเพียงอย่างเดียว ลักษณะเหมือนกับเวลาเรารักษาดุลของไม้กวาดบนมือ 
เพียงแต่ปัญหาถูกจำกัดอยู่ในระนาบ 2 มิติ รายละเอียดเพิ่มเติมของพลวัตของคาร์ทโพลสามารถศึกษาได้จาก
[Section 3.2 of Underactuated Robotics](https://underactuated.csail.mit.edu/acrobot.html#cart_pole) 
<div align="center">
<img src="https://raw.githubusercontent.com/dewdotninja/sharing-github/master/cart_pole.png" width=500 />
</div>
<div align="center">รูปที่ C.1 ระบบคาร์ทโพลในระนาบ 2 มิติ</div>

นำเข้าแพ็กเกจที่ต้องการใช้งาน

In [63]:
import numpy as np 
from numpy.linalg import norm
import matplotlib.pyplot as plt 
import control as ctl
from pydrake import forwarddiff
from pydrake.autodiffutils import AutoDiffXd, ExtractGradient, ExtractValue, InitializeAutoDiff

เพื่อทำให้ปัญหาง่ายขึ้น แทนค่าพารามิเตอร์ดังนี้

- มวลของรถเข็น $m_c=1$,
- มวลของลูกตุ้ม $m_p=1$,
- ความยาวของเสาลูกตุ้ม $l=1$,
- แรงโน้มถ่วงโลก $g=9.81$.

สถานะของคาร์ทโพลนิยามได้เป็น $\mathbf{x} = [x, \theta, \dot{x}, \dot{\theta}]^T$ 
และแรงที่กระทำกับรถเข็นคืออินพุตควบคุม $\mathbf{u} = [f_x]$

ใช้ [สมการ (16) และ (17)](https://underactuated.csail.mit.edu/acrobot.html#cart_pole) จาก [3]
ที่บรรยายความเร่งเชิงเส้นและเชิงมุมของคาร์ทโพล

$$
\ddot{x} = \frac{1}{m_c + m_p \sin^2\theta}
[ f_x+m_p \sin\theta (l \dot\theta^2 + g\cos\theta)] \tag{C.8}
$$

$$
\ddot{\theta} = \frac{1}{l(m_c + m_p \sin^2\theta)}
[ -f_x \cos\theta - m_p l \dot\theta^2 \cos\theta \sin\theta - (m_c + m_p) g \sin\theta] \tag{C.9}
$$

อิมพลิเมนต์เวกเตอร์
$\dot{\mathbf{x}} = [\dot{x}, \dot{\theta}, \ddot{x}, \ddot{\theta}]^T$ 
เป็น $\dot{\mathbf{x}} = {\bf f}(\mathbf{x}, \mathbf{u})$. 
ในฟังก์ชัน ${\bf f}$ ด้านล่างเพื่อคืนค่า $\dot{\mathbf{x}}$


In [56]:
# function that given the cart-pole state x (4d array)
# and the input u (1d array) returns the right
# hand side of the state space dynamics h(x,u)
# (remember that we fixed the cart-pole parameters
# to the values above!)
def cartpole(x, u):
    c = np.cos(x[1])  # cos(theta)
    s = np.sin(x[1])  # sin(theta)
    # parameters
    m_c = 1.0  # cart mass
    m_p = 1.0  # pendulum mass
    l = 1.0  # pole length
    g = 9.81  # gravity

    f1 = x[2]  # x_dot is the 3rd element of state vector
    f2 = x[3]  # theta_dot is the 4th element of state vector
    f3 = (1/(m_c + m_p*s**2))*(u + m_p*s*(l*x[3]**2+g*c))  # from (16) of [3]
    f4 = (1/(l*(m_c + m_p*s**2)))*(-u*c - m_p*l*x[3]**2*c*s - (m_c+m_p)*g*s)  # from (17) of [3]

    return np.array([f1, f2, f3, f4]).reshape(4,1)

ต้องการแปลงเป็นเชิงเส้นรอบจุดสมดุลที่ไม่เสถียร 
พิจารณาสถานะสมดุลที่ไม่เสถียร 
$$\mathbf{x}^* = [0, \pi, 0, 0]^T \tag{C.10}
$$
โดยมีอินพุตควบคุม ณ จุดสมดุลนี้เป็น
$$
\mathbf{u}^* = [0] \tag{C.11}
$$

ใช้วิธีการที่กล่าวถึงในหนังสือ
[[3]](https://underactuated.csail.mit.edu/acrobot.html#linearizing_manip)
ในการอนุพัทธ์โมเดลเชิงเส้นในรูป
$$\dot{\bar{\mathbf{x}}} = A_{\text{lin}} \mathbf{\bar{x}} + B_{\text{lin}} \mathbf{\bar{u}},$$
where $\mathbf{\bar{x}} = \mathbf{x}-\mathbf{x}^*$ and $\mathbf{\bar{u}} = \mathbf{u} -\mathbf{u}^*$.

รายละเอียดแสดงในรูปที่ C.2                          
<div align="center">
<img src="https://raw.githubusercontent.com/dewdotninja/sharing-github/master/cardpole_linearize.png" width=600 />
</div>
<div align="center">
รูปที่ C.2 รายละเอียดการแปลงคาร์ทโพลเป็นระบบเชิงเส้น
</div>

สร้างเมทริกซ์ $A_{c1}$ และเวกเตอร์ $B_{c1}$
ตามรายละเอียดในรูปที่ C.2 

In [6]:
g = 9.81  # gravity
A_c1 = np.array(
    [
        [0, 0, 1, 0],  
        [0, 0, 0, 1],  
        [0, g, 0, 0], 
        [0, 2*g, 0, 0],  
    ]
)
B_c1 = np.array(
    [
        [0],
        [0],
        [1],
        [1],
    ]  
)


แปลงเป็นระบบดีสครีต $A_{d1}, B_{d1}$ โดยใช้แพ็กเกจ Python Control 
เลือกคาบเวลา $h = 0.01$ วินาที

In [14]:
h = 0.01 # period (sec)
C_c1 = np.array([1,0,0,0])
sys_c = ctl.ss(A_c1,B_c1,C_c1,0)
sys_d = ctl.sample_system(sys_c,h)
A_d1,B_d1,C_d1,D_d1 = ctl.ssdata(sys_d)


สำหรับวิธีการประมาณค่าเชิงเส้นระบบดีสครีตโดยใช้การหาอนุพันธ์อัตโนมัติ เริ่มจากการนิยามฟังก์ชัน 
สำหรับการหาปริพันธ์โดยวิธีรุงเงอ-คุตตา 

In [44]:
def cartpole_rk4(x,u):
    global f1, f2, f3, f4, ft,x_1,x_a
    #RK4 integration with zero-order hold on u
    x_1 = x
    f1 = cartpole(x, u)
    
    x_a = x.reshape(4,1)
    x_2 = x_a + 0.5*h*f1
    f2 = cartpole(x_2.flatten(), u)
    
    x_3 = x_a + 0.5*h*f2
    f3 = cartpole(x_3.flatten(), u)

    x_4 = x_a + 0.5*h*f3
    f4 = cartpole(x_4.flatten(), u)

    ft = x_a + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    
    return ft

สร้างฟังก์ชันห่อหุ้มและฟังก์ชันหาอนุพันธ์เทียบ $x$ และ $u$ ตามที่ได้อธิบายในหัวข้อก่อนหน้านี้ โดยสำหรั้บ 
dxdu() จะใช้ฟังก์ชัน vectorout_derivative() ตามที่ได้อธิบายในหัวข้อ C.1.1

In [26]:
def vectorout_derivative(function, x):
    """ origimal comment from forwarddiff.derivative()
    Compute the derivative of the function evaluated at the scalar input x
    using Eigen's automatic differentiation.

    The function should be scalar-input and scalar-output.
    """
    x_ad = AutoDiffXd(value=x, size=1, offset=0)
    y_ad = function(x_ad)
    y_derivative = ExtractGradient(y_ad) # changed
    return y_derivative

In [17]:
# wrapper and forwarddiff functions
def cartpole_rk4_x(x):
    global u_global 
    return cartpole_rk4(x, u_global)

def cartpole_rk4_u(u):
    global x_global 
    return cartpole_rk4(x_global.flatten(), u)    
    
def cartpole_dfdx(x,u):
    global adflag
    adflag = 1
    return np.squeeze(forwarddiff.jacobian(cartpole_rk4_x,x.flatten()))

def cartpole_dfdu(x,u):
    global adflag
    adflag = 2
    return vectorout_derivative(cartpole_rk4_u,u)
    

กำหนดจุดสมดุลตาม (C.9), (C.10)

In [30]:
x_star = np.array([0, np.pi, 0, 0])
u_star = 0.0

ใช้่การหาอนุพันธ์อัตโนมัติเพื่อประมาณค่าโมเดลเชิงเส้น

In [57]:
u_global = u_star
A_d2 = cartpole_dfdx(x_star,u_star)
A_d2

array([[1.00000000e+00, 4.08790098e-04, 1.00000000e-02, 1.22625000e-06],
       [0.00000000e+00, 1.00081758e+00, 0.00000000e+00, 1.00024525e-02],
       [0.00000000e+00, 9.81240590e-02, 1.00000000e+00, 4.08790098e-04],
       [0.00000000e+00, 1.96248118e-01, 0.00000000e+00, 1.00081758e+00]])

In [58]:
x_global = x_star
B_d2 = cartpole_dfdu(x_star,u_star)
B_d2

array([[4.16687104e-05],
       [4.16707542e-05],
       [1.00012263e-02],
       [1.00024525e-02]])

เขียนฟังก์ชันเพื่อเปรียบเทียบสถานะ $x_{k+1} = Ax_k + Bu_k$
จากการโมเดลบนกระดาษกับการคำนวณอนุพันธ์อัตโนมัติ 
โดยให้ $x_k, u_k$ เป็นค่าสุ่ม 

In [59]:
def cartpole_linear(A,B,x,u):
    x_bar = x - x_star
    u_bar = u - u_star
    return A.dot(x_bar) + B.dot(u_bar) 

def linearized_difference(x,u):
    return norm(cartpole_linear(A_d1,B_d1,x,u) - cartpole_linear(A_d2,B_d2,x,u))
    

ทดลองรันเซลล์นี้หลายครั้งเพื่อดูผลการเปรียบเทียบ ค่าความแตกต่างจะต้องมีค่าน้อย

In [69]:
x_test = np.random.rand(4,1)
u_test = np.random.rand()
linearized_difference(x_test,u_test)

np.float64(0.0008046064602814321)

### C.1.4 คำนวณอนุพันธ์ผ่านฟังก์ชันรุงเงอ คุตตา

ในการหาโมเดลระบบดีสครีตของคาร์ทโพลในหัวข้อ C.1.3 เราใช้การหาอนุพันธ์อัตโนมัติผ่านฟังก์ชันรุงเงอ 
คุตตา (RK4) ซึ่งในหนังสือนี้จะใช้วิธีการนี้อยู่ตลอด 
ดังนั้นในหัวข้อนี้เราจะตรวจสอบการคำนวณอนุพันธ์อัตโนมัติผ่าน RK4 
เปรียบเทียบผลกับการคำนวณโดยเขียนฟังก์ชันเอง 
โดยเมื่อเข้าใจแล้วเราสามารถเพิ่มการคำนวณอนุพันธ์ด้วยตัวเองในส่วนที่แพ็กเกจไม่สามารถทำได้ 

**ตัวอย่าง C.8** ตัวอย่างนี้จะขยายจากโจทย์เดิมในตัวอย่าง C.3 พิจารณาฟังก์ชันไม่เป็นเชิงเส้น 

$$
f(x) =
\left[\begin{array}{c}
f_1(x) \\
f_2(x)
\end{array}\right] =
\left[\begin{array}{c}
x_1^3  + 2x_2 \\
x_1cos(x_2)
\end{array}\right] \tag{C.13}
$$

คำนวณจาโคเบียนได้เท่ากับ
$$
\left[\begin{array}{cc}
\frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} \\
\frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2} \\
\end{array}\right] = 
\left[\begin{array}{cc}
3x_1^2 & 2 \\
cos(x_2) & -x_1sin(x_2) \\
\end{array}\right] \tag{C.14}
$$
 

In [1]:
import numpy as np
from pydrake import forwarddiff
from pydrake.autodiffutils import AutoDiffXd, ExtractGradient, ExtractValue, InitializeAutoDiff

In [9]:
def f(x):
    f_1 = x[0]**3  + 2*x[1]
    f_2 = x[0]*np.cos(x[1])
    return np.array([f_1,f_2])

เขียนฟังก์ชัน rk4_f() เพื่อหาคำตอบโดยใช้การคำนวณปริพันธ์แบบ RK4 กำหนดค่าขั้น h = 0.05

In [7]:
h  = 0.05
def rk4_f(x):
    #RK4 integration 
    f1 = f(x)

    x_2 = x + 0.5*h*f1
    f2 = f(x_2)
    
    x_3 = x + 0.5*h*f2
    f3 = f(x_3)

    x_4 = x + 0.5*h*f3
    f4 = f(x_4)

    ft = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    
    return ft

คำนวณอนุพันธ์ที่อินพุต $x = [1, \pi /2]$ โดยใช้ forwarddiff.jacobian 

In [10]:
x = np.array([1.0, np.pi/2])
y_ad = forwarddiff.jacobian(rk4_f,x)
print(y_ad)

[[1.19249094e+00 1.05820619e-01]
 [3.21728845e-18 9.46673632e-01]]


ในการศึกษาการคำนวณอนุพันธ์อัตโนมัติผ่าน RK4 เริ่มจากเขียนฟังก์ชัน df() 
เพื่อคำนวณอนุพันธ์ด้วยตัวเองตาม (C.14)

In [11]:
def jacobian_f(x):
    m_00 = 3*x[0]**2
    m_01 = 2
    m_10 = np.cos(x[1])
    m_11 = -x[0]*np.sin(x[1])
    M = np.array([[m_00, m_01],[m_10, m_11]])
    return M

โดยจากการตรวจสอบค่าของเอาต์พุตฟังก์ชันและอนุพันธ์ที่ได้จากการเรียกฟังก์ขัน f() 
ผ่าน forwarddiff ในแต่ละครั้ง เราเขียนฟังก์ชัน jacobian_rk4_f() ได้ดังนี้

In [12]:
def jacobian_rk4_f(x):
    # manuallydo RK4 on derivatives
    # 1. you must start initial xd as np.eye(2)
    # 2. you must keep track of changes in x value and evaluate 
    # derivative at that point!
    
    xd_1 = np.eye(2) # initial xd
    f1 = f(x)  # function value
    df1 = jacobian_f(x) # df1 is 2x2 matrix

    x_2 = x + 0.5*h*f1 # update x for 2nd f() call
    xd_2 = xd_1 + 0.5*h*df1 # update Jacobian
    f2 = f(x_2)
    df2 = jacobian_f(x_2)

    x_3 = x + 0.5*h*f2
    xd_3 = xd_1 + 0.5*h*df2
    f3 = f(x_3)
    df3 = jacobian_f(x_3)

    x_4 = x + 0.5*h*f3

    xd_4 = xd_1 + 0.5*h*df3
    f4 = f(x_4)
    df4 = jacobian_f(x_4)

    dft = xd_1 + (h/6.0)*(df1 + 2*df2 + 2*df3 + df4)
    
    return dft 

อธิบายการคำนวณอนุพันธ์ผ่าน RK4 ในฟังก์ชัน jacobian_rk4_f() ได้ดังนี้ 
ในแต่ละครั้งของการเรียก f() 

1. คำนวณเอาต์พุตของฟังก์ชันเพื่อใช้ในการอัพเดตค่า x ในครั้งต่อไป 
2. คำนวณค่าอนุพันธ์ในแต่ละครั้งโดยใช้ค่า x ที่อัพเดตจากข้อ 1 โดยเป็นค่าเมทริกซ์จาโคเบียน 
สังเกตว่าอนุพันธ์เริ่มต้นที่ตำแหน่ง x คือ

$$
\left[\begin{array}{cc}
\frac{\partial x_1}{\partial x_1} & \frac{\partial x_2}{\partial x_1} \\
\frac{\partial x_2}{\partial x_1} & \frac{\partial x_2}{\partial x_2}
\end{array}\right] = 
\left[\begin{array}{cc}
1 & 0 \\
0 & 1
\end{array}\right] 
$$

ในขั้นตอนสุดท้าย เฉลี่ยค่าเมทริกซ์จาโคเบียนทั้ง 4 ครั้งตามสูตรของ RK4 ได้เป็นค่าเอาต์พุต
    
เมื่อคำนวณจาโคเบียนผ่าน RK4 ด้วยตัวเอง ได้ผลดังนี้

In [13]:
y_md = jacobian_rk4_f(x)
y_md

array([[1.17869770e+00, 1.00000000e-01],
       [3.06161700e-18, 9.45464314e-01]])

จะเห็นว่าได้ผลใกล้เคียงกับการคำนวณอัตโนมัติ 
ในเซลล์ด้านล่างนี้เป็นการเปรียบเทียบโดยใช้อินพุต x เป็นค่าสุ่ม ทดลองรันเซลล์นี้หลายครั้ง 
แต่ละครั้งความแตกต่างต้องมีค่าน้อย

In [17]:
x_test = np.random.rand(2,)
y_ad = forwarddiff.jacobian(rk4_f,x_test)
y_md = jacobian_rk4_f(x_test)
np.linalg.norm(y_ad - y_md)

np.float64(0.002927749041612254)

### C.1.5 คำนวณเฮสเซียนเชิงสัญลักษณ์

วิธีการ DDP ในบทที่ 5  มีพจน์เฮสเซียนในการคำนวณย้อนหลัง ซึ่งแพ็กเกจ Drake ไม่มีฟังก์ชันสนับสนุน 
และไม่สามารถหาวิธีการแก้ปัญหาโดยใช้ฟังก์ชันที่มีอยู่ได้ สำหรับแพ็กเกจ JAX แม้จะมีฟังก์ชันคำนวณพจน์เฮสเซียนได้ 
แต่พบว่าทำงานช้ามาก จึงแนะนำวิธีการ iLOR โดยใช้อัลกอริทึมเกาส์-นิวตันที่ละทิ้งพจน์เฮสเซียนไป จะทำให้ได้คำตอบแนววิถีเหมาะที่สุดเร็วกว่า 

อย่างไรก็ตามหากต้องการคำนวณพจน์เฮสเซียนที่ใช้ในอัลกอริทึมนิวตันแบบเต็ม ในหัวข้อนี้เสนอทางเลือกคือใช้แพ็กเกจ SymPy 
หาคำตอบของเฮสเซียนเชิงสัญลักษณ์ แล้วอิมพลิเมนต์เป็นฟังก์ชันเพื่อให้ทำงานเร็วขึ้น 

**ตัวอย่าง C.9**  จากพลวัตของคาร์ทโพลในตัวอย่าง C.7 เราปรับพารามิเตอร์เพื่อให้สอดคล้องกับตัวอย่าง 5.2 ในบทที่ 5 
ในตัวอย่างนี้เราจะคำนวณพจน์เฮสเซียนโดยวิธีเชิงสัญลักษณ์ และเปรียบเทียบกับการคำนวณโดย JAX 

นำเข้าแพ็กเกจทั้งหมดรวมทั้ง SymPy สำหรับหาอนุพันธ์เชิงสัญลักษณ์

In [1]:
import numpy as np 
from numpy import kron
import matplotlib.pyplot as plt 
from scipy.linalg import block_diag
import jax
import jax.numpy as jnp
from jax.numpy.linalg import solve, eigvals
from sympy import symbols,sin,cos,Array,diff,simplify,lambdify

นิยามฟังก์ชันทั้งหมดที่ต้องใช้ดังเช่นในตัวอย่าง 5.2 (เพื่อให้โค้ดดูง่ายขึ้นจะคงไว้เฉพาะกรณีที่ใช้ JAX คือเรียก jax.numpy แทน numpy)

In [2]:
def cartpole(x, u):

    c = jnp.cos(x[1])  # cos(theta)
    s = jnp.sin(x[1])  # sin(theta)

    # parameters. Changed to match RobotZoo's cartpole
    m_c = 1.0  # cart mass
    m_p = 0.2 # pendulum mass
    l = 0.5  # pole length
    g = 9.81  # gravity

    f1 = x[2]  # x_dot is the 3rd element of state vector
    f2 = x[3]  # theta_dot is the 4th element of state vector
    f3 = (1/(m_c + m_p*s**2))*(u + m_p*s*(l*x[3]**2+g*c))   
    f4 = (1/(l*(m_c + m_p*s**2)))*(-u*c - m_p*l*x[3]**2*c*s - (m_c+m_p)*g*s)  
    y = jnp.array([f1, f2, f3, f4]).reshape(4,1)
    return y

In [3]:
h = 0.05 # time step

def cartpole_rk4(x,u):
    global f1, f2, f3, f4, ft,x_1,x_a
    #RK4 integration with zero-order hold on u
    x_1 = x
    f1 = cartpole(x, u)
    
    x_a = x.reshape(4,1)
    x_2 = x_a + 0.5*h*f1
    f2 = cartpole(x_2.flatten(), u)
    
    x_3 = x_a + 0.5*h*f2
    f3 = cartpole(x_3.flatten(), u)

    x_4 = x_a + 0.5*h*f3
    f4 = cartpole(x_4.flatten(), u)

    ft = x_a + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    
    return ft


In [4]:
# convert [m,n,p] tensor to [m*n,p] matrix
def tensor2mat(Mat3d):
    dim3 = Mat3d.shape[2]
    M = Mat3d[:,:,0]
    for i in range(dim3-1):
        M = jnp.vstack((M,Mat3d[:,:,i+1]))
    return M

โค้ดในส่วนการคำนวณเกรเดียนต์และเฮสเซียน

In [5]:
def dfdx(x,u):
    y = jnp.squeeze(jax.jacfwd(cartpole_rk4,argnums=0)(x,u))
    return y

def dfdu(x,u):
    y = jax.jacfwd(cartpole_rk4,argnums=1)(x,u)
    return y


def dAdx(x,u):
    y = jax.hessian(cartpole_rk4)(x,u)
    return tensor2mat(y)[:,0,:]

def dBdx(x,u):
    y = jax.jacfwd(dfdu)(x,u)
    return jnp.squeeze(y)

def dAdu(x,u):
    y = jax.jacfwd(dfdu)(x,u)
    return tensor2mat(y)

def dBdu(x,u):
    y = jax.jacfwd(dfdu,argnums=1)(x,u)
    return y


คำนวณคำตอบเมื่อกำหนดค่าเวกเตอร์ x_star และสเกลาร์ u_star

In [6]:
x_star = np.array([0, np.pi, 0, 0])
u_star = 1.0
Ax_ad = dAdx(x_star,u_star)
print("Ax shape = "+str(Ax_ad.shape))
Ax_ad

Ax shape = (16, 4)


Array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00, -4.2999975e-04,  0.0000000e+00, -1.2782900e-05],
       [ 0.0000000e+00, -2.9712569e-03,  0.0000000e+00, -5.7176469e-05],
       [ 0.0000000e+00, -2.1210689e-02,  0.0000000e+00, -8.6591602e-04],
       [ 0.0000000e+00, -1.4577442e-01,  0.0000000e+00, -3.8740120e-03],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00, -1.2782899e-05,  0.0000000e+00, -5.2559028e-07],
       [ 0.0000000e+00, -5.7176461e-05,  0.0000000e

In [7]:
Bx_ad = dBdx(x_star,u_star)
print("Bx shape = "+str(Bx_ad.shape))
Bx_ad

Bx shape = (4, 4)


Array([[ 0.0000000e+00, -5.3190615e-07,  0.0000000e+00, -1.8390034e-08],
       [ 0.0000000e+00, -2.1313697e-06,  0.0000000e+00, -6.3080179e-08],
       [ 0.0000000e+00, -4.7298545e-05,  0.0000000e+00, -1.8012237e-06],
       [ 0.0000000e+00, -2.2279301e-04,  0.0000000e+00, -6.7763485e-06]],      dtype=float32)

In [8]:
Au_ad = dAdu(x_star,u_star)
print("Au shape = "+str(Au_ad.shape))
Au_ad

Au shape = (16, 1)


Array([[ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [-5.3190615e-07],
       [-2.1313697e-06],
       [-4.7298545e-05],
       [-2.2279301e-04],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [-1.8390034e-08],
       [-6.3080179e-08],
       [-1.8012237e-06],
       [-6.7763485e-06]], dtype=float32)

In [9]:
Bu_ad = dBdu(x_star,u_star)
print("Bu shape = "+str(Bu_ad.shape))
Bu_ad

Bu shape = (4, 1)


Array([[-7.8545476e-10],
       [-3.5369256e-09],
       [-9.4716206e-08],
       [-4.2535433e-07]], dtype=float32)

เราจะทดลองคำนวณค่าทั้งสี่ตัวนี้เชิงสัญลักษณ์ แปลงเป็นฟังก์ชันเพื่อให้ทำงานเร็วแล้วเปรียบเทียบกัน 
หากได้ค่าที่ไม่แตกต่างกันมากก็จะทดลองใช้ในการหาแนววิถีเหมาะที่สุดโดยวิธี DDP 
ในกรณีที่ใช้ Drake และไม่สามารถคำนวณเฮสเชียนได้โดยฟังก์ชันจากแพ็กเกจ

เริ่มจากกรณี dAdx พิจารณาพลวัตของคาร์ทโพลเมื่อเขียนในรูป

$$
\left[\begin{array}{c}
\dot{x}_1 \\
\dot{x}_2 \\
\dot{x}_3 \\
\dot{x}_4 
\end{array}\right] = 
\left[\begin{array}{c}
f_1(x) \\
f_2(x) \\
f_3(x) \\
f_4(x) 
\end{array}\right], \;\;\; x = 
\left[\begin{array}{c}
x_1 \\
x_2 \\
x_3 \\
x_4 
\end{array}\right] \tag{C.15}
$$

สามารถแสดงได้ว่าเฮสเซียนเมื่อแปลงจากเทนเซอร์เป็นเมทริกซ์แล้วจะเขียนได้อยู่ในรูป
$$
\frac{\partial A}{\partial x} = 
\left[\begin{array}{cccc}
\frac{\partial^2 f_1}{\partial x_1^2} & \frac{\partial^2 f_1}{\partial x_1 \partial x_2} &
\frac{\partial^2 f_1}{\partial x_1 \partial x_3} & \frac{\partial^2 f_1}{\partial x_1 \partial x_4} \\
\frac{\partial^2 f_2}{\partial x_1^2} & \frac{\partial^2 f_2}{\partial x_1 \partial x_2} &
\frac{\partial^2 f_2}{\partial x_1 \partial x_3} & \frac{\partial^2 f_2}{\partial x_1 \partial x_4} \\
\frac{\partial^2 f_3}{\partial x_1^2} & \frac{\partial^2 f_3}{\partial x_1 \partial x_2} &
\frac{\partial^2 f_3}{\partial x_1 \partial x_3} & \frac{\partial^2 f_3}{\partial x_1 \partial x_4} \\
\frac{\partial^2 f_4}{\partial x_1^2} & \frac{\partial^2 f_4}{\partial x_1 \partial x_2} &
\frac{\partial^2 f_4}{\partial x_1 \partial x_3} & \frac{\partial^2 f_4}{\partial x_1 \partial x_4} \\
\frac{\partial^2 f_1}{\partial x_1 \partial x_2} & \frac{\partial^2 f_1}{\partial x_2^2} &
\frac{\partial^2 f_1}{\partial x_2 \partial x_3} & \frac{\partial^2 f_1}{\partial x_2 \partial x_4} \\
\frac{\partial^2 f_2}{\partial x_1 \partial x_2} & \frac{\partial^2 f_2}{\partial x_2^2} &
\frac{\partial^2 f_2}{\partial x_2 \partial x_3} & \frac{\partial^2 f_2}{\partial x_2 \partial x_4} \\
\frac{\partial^2 f_3}{\partial x_1 \partial x_2} & \frac{\partial^2 f_3}{\partial x_2^2} &
\frac{\partial^2 f_3}{\partial x_2 \partial x_3} & \frac{\partial^2 f_3}{\partial x_2 \partial x_4} \\
\frac{\partial^2 f_4}{\partial x_1 \partial x_2} & \frac{\partial^2 f_4}{\partial x_2^2} &
\frac{\partial^2 f_4}{\partial x_2 \partial x_3} & \frac{\partial^2 f_4}{\partial x_2 \partial x_4} \\
\frac{\partial^2 f_1}{\partial x_1 \partial x_3} & \frac{\partial^2 f_1}{\partial x_2 \partial x_3} &
\frac{\partial^2 f_1}{\partial x_3^2} & \frac{\partial^2 f_1}{\partial x_3 \partial x_4} \\
\frac{\partial^2 f_2}{\partial x_1 \partial x_3} & \frac{\partial^2 f_2}{\partial x_2 \partial x_3} &
\frac{\partial^2 f_2}{\partial x_3^2} & \frac{\partial^2 f_2}{\partial x_3 \partial x_4} \\
\frac{\partial^2 f_3}{\partial x_1 \partial x_3} & \frac{\partial^2 f_3}{\partial x_2 \partial x_3} &
\frac{\partial^2 f_3}{\partial x_3^2} & \frac{\partial^2 f_3}{\partial x_3 \partial x_4} \\
\frac{\partial^2 f_4}{\partial x_1 \partial x_3} & \frac{\partial^2 f_4}{\partial x_2 \partial x_3} &
\frac{\partial^2 f_4}{\partial x_3^2} & \frac{\partial^2 f_4}{\partial x_3 \partial x_4} \\
\frac{\partial^2 f_1}{\partial x_1 \partial x_4} & \frac{\partial^2 f_1}{\partial x_2 \partial x_4} &
\frac{\partial^2 f_1}{\partial x_3 \partial x_4} & \frac{\partial^2 f_1}{\partial x_4^2} \\
\frac{\partial^2 f_2}{\partial x_1 \partial x_4} & \frac{\partial^2 f_2}{\partial x_2 \partial x_4} &
\frac{\partial^2 f_2}{\partial x_3 \partial x_4} & \frac{\partial^2 f_2}{\partial x_4^2} \\
\frac{\partial^2 f_3}{\partial x_1 \partial x_4} & \frac{\partial^2 f_3}{\partial x_2 \partial x_4} &
\frac{\partial^2 f_3}{\partial x_3 \partial x_4} & \frac{\partial^2 f_3}{\partial x_4^2} \\
\frac{\partial^2 f_4}{\partial x_1 \partial x_4} & \frac{\partial^2 f_4}{\partial x_2 \partial x_4} &
\frac{\partial^2 f_4}{\partial x_3 \partial x_4} & \frac{\partial^2 f_4}{\partial x_4^2} 
\end{array}\right] \tag{C.16}    
$$

สร้างพลวัตของคาร์ทโพลเชิงสัญลักษณ์โดยใช้แพ็กเกจ SymPy

In [10]:
#from sympy import *  # already imorted above
x1,x2,x3,x4,u = symbols("x1,x2,x3,x4,u")

In [11]:
m_c = 1.0  # cart mass
m_p = 0.2 # pendulum mass
l = 0.5  # pole length
g = 9.81  # gravity

In [12]:
s2 = sin(x2)
c2 = cos(x2)
f1 = x3
f2 = x4
f3 = (1/(m_c + m_p*s2**2))*(u + m_p*s2*(l*x4**2+g*c2))
f4 = (1/(l*(m_c + m_p*s2**2)))*(-u*c2 - m_p*l*x4**2*c2*s2 - (m_c+m_p)*g*s2)
cartpole_sym = Array([[f1],[f2],[f3],[f4]])
cartpole_sym

[[x3], [x4], [(u + 0.2*(0.5*x4**2 + 9.81*cos(x2))*sin(x2))/(0.2*sin(x2)**2 + 1.0)], [(-u*cos(x2) - 0.1*x4**2*sin(x2)*cos(x2) - 11.772*sin(x2))/(0.1*sin(x2)**2 + 0.5)]]

คำนวณสมาชิกของเมทริกซ์เฮสเชียน (C.16) 
สังเกตว่าสมาชิกบางตำแหน่งมีค่าเท่ากันเช่น $\frac{\partial^2 f_i}{\partial x_j \partial x_k} = \frac{\partial^2 f_i}{\partial x_k \partial x_j}$

In [13]:
# elements of Hessian
d2f1_x1sq = diff(cartpole_sym[0],x1,x1)
d2f1_x3sq = diff(cartpole_sym[0],x3,x3)
d2f1_x4sq = diff(cartpole_sym[0],x4,x4)
d2f1_x1x2 = diff(cartpole_sym[0],x1,x2)
d2f1_x1x3 = diff(cartpole_sym[0],x1,x3)
d2f1_x1x4 = diff(cartpole_sym[0],x1,x4)
d2f1_x2sq = diff(cartpole_sym[0],x2,x2)
d2f1_x2x3 = diff(cartpole_sym[0],x2,x3)
d2f1_x3x4 = diff(cartpole_sym[0],x3,x4)
d2f1_x2x4 = diff(cartpole_sym[0],x2,x4)

d2f2_x1sq = diff(cartpole_sym[1],x1,x1)
d2f2_x2x1 = diff(cartpole_sym[1],x2,x1)
d2f2_x2sq = diff(cartpole_sym[1],x2,x2)
d2f2_x3sq = diff(cartpole_sym[1],x3,x3)
d2f2_x4sq = diff(cartpole_sym[1],x4,x4)
d2f2_x2x3 = diff(cartpole_sym[1],x2,x3)
d2f2_x2x4 = diff(cartpole_sym[1],x2,x4)
d2f2_x1x3 = diff(cartpole_sym[1],x1,x3)
d2f2_x1x4 = diff(cartpole_sym[1],x1,x4)
d2f2_x3x4 = diff(cartpole_sym[1],x3,x4)

d2f3_x3x1 = diff(cartpole_sym[2],x3,x1)
d2f3_x3x2 = diff(cartpole_sym[2],x3,x2)
d2f3_x3sq = diff(cartpole_sym[2],x3,x3)
d2f3_x3x4 = diff(cartpole_sym[2],x3,x4)
d2f3_x1sq = diff(cartpole_sym[2],x1,x1)
d2f3_x2sq = diff(cartpole_sym[2],x2,x2)
d2f3_x4sq = diff(cartpole_sym[2],x4,x4)
d2f3_x1x2 = diff(cartpole_sym[2],x1,x2)
d2f3_x1x4 = diff(cartpole_sym[2],x1,x4)
d2f3_x2x4 = diff(cartpole_sym[2],x2,x4)

d2f4_x4x1 = diff(cartpole_sym[3],x4,x1)
d2f4_x4x2 = diff(cartpole_sym[3],x4,x2)
d2f4_x4x3 = diff(cartpole_sym[3],x4,x3)
d2f4_x4sq = diff(cartpole_sym[3],x4,x4)
d2f4_x2sq = diff(cartpole_sym[3],x2,x2)
d2f4_x1sq = diff(cartpole_sym[3],x1,x1)
d2f4_x3sq = diff(cartpole_sym[3],x3,x3)
d2f4_x1x2 = diff(cartpole_sym[3],x1,x2)
d2f4_x1x3 = diff(cartpole_sym[3],x1,x3)
d2f4_x2x3 = diff(cartpole_sym[3],x2,x3)

เมื่อตรวจสอบคำตอบเราพบว่ามีสมาชิกเพียงบางตัวที่มีค่าไม่เป็นศูนย์ 
เราอาจใช้ฟังก์ชัน simplify() สำหรับสมาชิกนั้นเพื่อทำให้พจน์ง่ายขึ้น 

In [14]:
Ax_sym = Array([
    [d2f1_x1sq, d2f1_x1x2, d2f1_x1x3, d2f1_x1x4],
    [d2f2_x1sq, d2f2_x2x1, d2f2_x1x3, d2f2_x1x4],
    [d2f3_x1sq, d2f3_x1x2, d2f3_x3x1, d2f3_x1x4],
    [d2f4_x1sq, d2f4_x1x2, d2f4_x1x3, d2f4_x4x1],

    [d2f1_x1x2, d2f1_x2sq, d2f1_x2x3, d2f1_x2x4],
    [d2f2_x2x1, d2f2_x2sq, d2f2_x2x3, d2f2_x2x4],
    [d2f3_x1x2, simplify(d2f3_x2sq), d2f3_x3x2, simplify(d2f3_x2x4)],
    [d2f4_x1x2, simplify(d2f4_x2sq), d2f4_x2x3, simplify(d2f4_x4x2)],

    [d2f1_x1x3, d2f1_x2x3, d2f1_x3sq, d2f1_x3x4],
    [d2f2_x1x3, d2f2_x2x3, d2f2_x3sq, d2f2_x3x4],
    [d2f3_x3x1, d2f3_x3x2, d2f3_x3sq, d2f3_x3x4],
    [d2f4_x1x3, d2f4_x2x3, d2f4_x3sq, d2f4_x4x3],
    
    [d2f1_x1x4, d2f1_x2x4, d2f1_x3x4, d2f1_x4sq],
    [d2f2_x1x4, d2f2_x2x4, d2f2_x3x4, d2f2_x4sq],
    [d2f3_x1x4, simplify(d2f3_x2x4), d2f3_x3x4, simplify(d2f3_x4sq)],
    [d2f4_x4x1, simplify(d2f4_x4x2), d2f4_x4x3, simplify(d2f4_x4sq)]
])[:,:,0]

In [15]:
Ax_sym

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, (-0.16*u*sin(x2)**4 + 1.04*u*sin(x2)**2 - 0.4*u - 0.036*x4**2*sin(x2)**5 - 0.008*x4**2*sin(x2)**3 - 0.008*x4**2*sin(x2)**2*sin(3*x2) - 0.1*x4**2*sin(x2) - 0.04*x4**2*sin(3*x2) + 1.72656*sin(x2)**3*cos(x2) - 10.2024*sin(x2)*cos(x2))/(0.2*sin(x2)**2 + 1.0)**3, 0, x4*(0.2 - 0.04*sin(x2)**2)*cos(x2)/(0.2*sin(x2)**2 + 1.0)**2], [0, ((1 - 0.0909090909090909*cos(2*x2))**2*(0.484*u*cos(3*x2) + 0.0363*x4**2*sin(4*x2) + 5.697648*sin(3*x2)) - 0.64*(0.1*sin(x2)**2 + 0.5)*(u*cos(x2) + 0.05*x4**2*sin(2*x2) + 11.772*sin(x2))*sin(x2)**2*cos(x2)**2 + (0.2*sin(x2)**2 + 1.0)**3*(u*cos(x2) + 0.2*x4**2*sin(2*x2) + 11.772*sin(x2)))/((0.1*sin(x2)**2 + 0.5)*(0.2*sin(x2)**2 + 1.0)**3), 0, 10.0*x4*(0.088*sin(x2)**2 - 0.04)/(0.2*sin(x2)**2 + 1.0)**2], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, x4*(0.2 - 0.04*sin(x2)**2)*cos(x2)/(0.2*sin(x2)**2 + 1.0)**2, 0, 0.2*sin(x2)/(0.2*sin(x2

แม้ว่าจะสามารถใช้เมทริกซ์เชิงสัญลักษณ์นี้เพื่อหาคำตอบโดยตรง แต่จะใช้เวลาการคำนวณมาก วิธีการที่ดีกว่าคือแปลงเป็นฟังก์ชัน 
ซึ่งเราไม่จำเป็นต้องโค้ดนิพจน์ของสมาชิกโดยตรง แต่สามารถใช้ฟังก์ชัน lambdify() จากแพ็กเกจ SymPy 
ที่ช่วยในการแปลงเป็นฟังก์ชันสำหรับอิมพลิเมนต์โดยอัตโนมัติ

In [16]:
dAdx_cartpole = lambdify((x1,x2,x3,x4,u), Ax_sym)

สังเกตว่า dAdx_cartpole() เป็นระบบเวลาต่อเนื่อง หากต้องการเปรียบเทียบผลกับคำตอบที่ได้จาก JAX 
เราต้องดำเนินการอีกขั้นหนึ่งคือคำนวณเฮสเชียนนี้ผ่านฟังก์ชันหาปริพันธ์ RK4 โดยใช้วิธีการที่คล้ายกับในหัวข้อ C.1.4 
เพียงแต่ในกรณีนี้เป็นเฮสเซียนแทนจาโคเบียน เขียนฟังก์ชันการคำนวณ dAdx_cartpole_rk4() ดังนี้

In [17]:
def dAdx_cartpole_rk4(x,u):
    # manually do RK4 on hessian
    # 1. you must start initial xdd as a zero matrix
    # 2. you must keep track of changes in x value and evaluate 
    # Hessian at that point!
    
    xdd_1 = np.zeros((16,4)) # initial Hessian elements are all zero!
    f1 = cartpole(x,u).flatten()  # value output of 
    d2f1 = dAdx_cartpole(x[0],x[1],x[2],x[3],u) # d2f1 is 16x4 matrix

    x_2 = x + 0.5*h*f1 # update x for 2nd cartpole() call
    f2 = cartpole(x_2,u).flatten()
    d2f2 = dAdx_cartpole(x_2[0],x_2[1],x_2[2],x_2[3],u)

    x_3 = x + 0.5*h*f2
    f3 = cartpole(x_3,u).flatten()
    d2f3 = dAdx_cartpole(x_3[0],x_3[1],x_3[2],x_3[3],u)

    x_4 = x + 0.5*h*f3
    f4 = cartpole(x_4,u).flatten()
    d2f4 = dAdx_cartpole(x_4[0],x_4[1],x_4[2],x_4[3],u)
    d2ft = xdd_1 + (h/6.0)*(d2f1 + 2*d2f2 + 2*d2f3 + d2f4)
    
    return d2ft 

คำนวณ dAdx เปรียบเทียบกับผลที่ได้จาก JAX

In [18]:
Ax_md = dAdx_cartpole_rk4(x_star,u_star)
Ax_md

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -2.03187999e-02,  0.00000000e+00,
        -4.17892837e-04],
       [ 0.00000000e+00, -1.41618403e-01,  0.00000000e+00,
        -8.35784447e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
      

ทดสอบโดยใช้ค่าสุ่ม รันเซลล์นี้หลายครั้ง ผลต่างจะต้องมีค่าน้อย

In [21]:
x_test = np.random.rand(4,)
u_test = np.random.rand()
Ax_ad_test = dAdx(x_test,u_test) # from JAX
Ax_md_test = dAdx_cartpole_rk4(x_test,u_test) # from symbolic
np.linalg.norm(Ax_md_test - Ax_ad_test)

np.float32(0.03713284)

ใช้วิธีการเช่นนี้กับฟังก์ชันที่เหลือคือ dBdx(), dAdu(), dBdu() 

$$
\frac{\partial B}{\partial x} = 
\left[\begin{array}{cccc}
\frac{\partial^2 f_1}{\partial x_1 \partial u} & \frac{\partial^2 f_1}{\partial x_2 \partial u} &
\frac{\partial^2 f_1}{\partial x_3 \partial u} & \frac{\partial^2 f_1}{\partial x_4 \partial u} \\
\frac{\partial^2 f_2}{\partial x_1 \partial u} & \frac{\partial^2 f_2}{\partial x_2 \partial u} &
\frac{\partial^2 f_2}{\partial x_3 \partial u} & \frac{\partial^2 f_2}{\partial x_4 \partial u} \\
\frac{\partial^2 f_3}{\partial x_1 \partial u} & \frac{\partial^2 f_3}{\partial x_2 \partial u} &
\frac{\partial^2 f_3}{\partial x_3 \partial u} & \frac{\partial^2 f_3}{\partial x_4 \partial u} \\
\frac{\partial^2 f_4}{\partial x_1 \partial u} & \frac{\partial^2 f_4}{\partial x_2 \partial u} &
\frac{\partial^2 f_4}{\partial x_3 \partial u} & \frac{\partial^2 f_4}{\partial x_4 \partial u} 
\end{array}\right] \tag{C.17}    
$$

$$
\frac{\partial A}{\partial u} = 
\left[\begin{array}{c}
\frac{\partial^2 f_1}{\partial x_1 \partial u} \\
\frac{\partial^2 f_2}{\partial x_1 \partial u} \\
\frac{\partial^2 f_3}{\partial x_1 \partial u} \\
\frac{\partial^2 f_4}{\partial x_1 \partial u} \\
\frac{\partial^2 f_1}{\partial x_2 \partial u} \\
\frac{\partial^2 f_2}{\partial x_2 \partial u} \\
\frac{\partial^2 f_3}{\partial x_2 \partial u} \\
\frac{\partial^2 f_4}{\partial x_2 \partial u} \\
\frac{\partial^2 f_1}{\partial x_3 \partial u} \\
\frac{\partial^2 f_2}{\partial x_3 \partial u} \\
\frac{\partial^2 f_3}{\partial x_3 \partial u} \\
\frac{\partial^2 f_4}{\partial x_3 \partial u} \\
\frac{\partial^2 f_1}{\partial x_4 \partial u} \\
\frac{\partial^2 f_2}{\partial x_4 \partial u} \\
\frac{\partial^2 f_3}{\partial x_4 \partial u} \\
\frac{\partial^2 f_4}{\partial x_4 \partial u} 
\end{array}\right] \tag{C.18}   
$$

$$
\frac{\partial B}{\partial u} = 
\left[\begin{array}{c}
\frac{\partial^2 f_1}{\partial u^2} \\
\frac{\partial^2 f_2}{\partial u^2} \\
\frac{\partial^2 f_3}{\partial u^2} \\
\frac{\partial^2 f_4}{\partial u^2} 
\end{array}\right] \tag{C.19}   
$$


In [22]:
# compute each element in C.16 - C.19.Some are equal.
d2f1_x1u = diff(cartpole_sym[0],x1,u)
d2f1_x2u = diff(cartpole_sym[0],x2,u)
d2f1_x3u = diff(cartpole_sym[0],x3,u)
d2f1_x4u = diff(cartpole_sym[0],x4,u)

d2f2_x1u = diff(cartpole_sym[1],x1,u)
d2f2_x2u = diff(cartpole_sym[1],x2,u)
d2f2_x3u = diff(cartpole_sym[1],x3,u)
d2f2_x4u = diff(cartpole_sym[1],x4,u)

d2f3_x1u = diff(cartpole_sym[2],x1,u)
d2f3_x2u = diff(cartpole_sym[2],x2,u)
d2f3_x3u = diff(cartpole_sym[2],x3,u)
d2f3_x4u = diff(cartpole_sym[2],x4,u)

d2f4_x1u = diff(cartpole_sym[3],x1,u)
d2f4_x2u = diff(cartpole_sym[3],x2,u)
d2f4_x3u = diff(cartpole_sym[3],x3,u)
d2f4_x4u = diff(cartpole_sym[3],x4,u)

d2f1_u2 = diff(cartpole_sym[0],u,u)
d2f2_u2 = diff(cartpole_sym[1],u,u)
d2f3_u2 = diff(cartpole_sym[2],u,u)
d2f4_u2 = diff(cartpole_sym[3],u,u)

In [23]:
Bx_sym = Array([
    [d2f1_x1u, d2f1_x2u, d2f1_x3u, d2f1_x4u],
    [d2f2_x1u, d2f2_x2u, d2f2_x3u, d2f2_x4u],
    [d2f3_x1u, d2f3_x2u, d2f3_x3u, d2f3_x4u],
    [d2f4_x1u, d2f4_x2u, d2f4_x3u, d2f4_x4u]])[:,:,0]

Au_sym = Array([
    [d2f1_x1u],
    [d2f2_x1u],
    [d2f3_x1u],
    [d2f4_x1u],
    [d2f1_x2u],
    [d2f2_x2u],
    [d2f3_x2u],
    [d2f4_x2u],
    [d2f1_x3u],
    [d2f2_x3u],
    [d2f3_x3u],
    [d2f4_x3u],
    [d2f1_x4u],
    [d2f2_x4u],
    [d2f3_x4u],
    [d2f4_x4u]])[:,:,0]
    
Bu_sym = Array([ 
    [d2f1_u2],
    [d2f2_u2],
    [d2f3_u2],
    [d2f4_u2]])[:,:,0]



In [24]:
Bx_sym

[[0, 0, 0, 0], [0, 0, 0, 0], [0, -0.4*sin(x2)*cos(x2)/(0.2*sin(x2)**2 + 1.0)**2, 0, 0], [0, (0.8*cos(x2)**2/(0.2*sin(x2)**2 + 1)**2 + 1/(0.1*sin(x2)**2 + 0.5))*sin(x2), 0, 0]]

In [25]:
Au_sym

[[0], [0], [0], [0], [0], [0], [-0.4*sin(x2)*cos(x2)/(0.2*sin(x2)**2 + 1.0)**2], [(0.8*cos(x2)**2/(0.2*sin(x2)**2 + 1)**2 + 1/(0.1*sin(x2)**2 + 0.5))*sin(x2)], [0], [0], [0], [0], [0], [0], [0], [0]]

In [26]:
Bu_sym

[[0], [0], [0], [0]]

**หมายเหตุ :** จากผลนี้จะเห็นว่า Bu_sym มีค่าเป็นศูนย์ตลอด ดังนั้นในการวนซ้ำในวิธี DDP เราอาจละทิ้งพจน์ที่ขึ้นกับ dBdu() เพื่อประหยัดเวลาการคำนวณ

In [27]:
# implementation
dBdx_cartpole = lambdify((x1,x2,x3,x4,u), Bx_sym)
dAdu_cartpole = lambdify((x1,x2,x3,x4,u), Au_sym)
dBdu_cartpole = lambdify((x1,x2,x3,x4,u), Bu_sym)

In [28]:
def dBdx_cartpole_rk4(x,u):   
    xdd_1 = np.zeros((4,4)) # initial Hessian elements are all zero!
    f1 = cartpole(x,u).flatten()  # value output of 
    d2f1 = dBdx_cartpole(x[0],x[1],x[2],x[3],u) # d2f1 is 16x4 matrix

    x_2 = x + 0.5*h*f1 # update x for 2nd cartpole() call
    print
    f2 = cartpole(x_2,u).flatten()
    d2f2 = dBdx_cartpole(x_2[0],x_2[1],x_2[2],x_2[3],u)

    x_3 = x + 0.5*h*f2
    f3 = cartpole(x_3,u).flatten()
    d2f3 = dBdx_cartpole(x_3[0],x_3[1],x_3[2],x_3[3],u)

    x_4 = x + 0.5*h*f3
    f4 = cartpole(x_4,u).flatten()
    d2f4 = dBdx_cartpole(x_4[0],x_4[1],x_4[2],x_4[3],u)
    d2ft = xdd_1 + (h/6.0)*(d2f1 + 2*d2f2 + 2*d2f3 + d2f4)    
    return d2ft 

def dAdu_cartpole_rk4(x,u):   
    xdd_1 = np.zeros((16,1)) # initial Hessian elements are all zero!
    f1 = cartpole(x,u).flatten()  # value output of 
    d2f1 = dAdu_cartpole(x[0],x[1],x[2],x[3],u) # d2f1 is 16x4 matrix

    x_2 = x + 0.5*h*f1 # update x for 2nd cartpole() call
    print
    f2 = cartpole(x_2,u).flatten()
    d2f2 = dAdu_cartpole(x_2[0],x_2[1],x_2[2],x_2[3],u)

    x_3 = x + 0.5*h*f2
    f3 = cartpole(x_3,u).flatten()
    d2f3 = dAdu_cartpole(x_3[0],x_3[1],x_3[2],x_3[3],u)

    x_4 = x + 0.5*h*f3
    f4 = cartpole(x_4,u).flatten()
    d2f4 = dAdu_cartpole(x_4[0],x_4[1],x_4[2],x_4[3],u)
    d2ft = xdd_1 + (h/6.0)*(d2f1 + 2*d2f2 + 2*d2f3 + d2f4)    
    return d2ft 

def dBdu_cartpole_rk4(x,u):   
    xdd_1 = np.zeros((4,1)) # initial Hessian elements are all zero!
    f1 = cartpole(x,u).flatten()  # value output of 
    d2f1 = dBdu_cartpole(x[0],x[1],x[2],x[3],u) # d2f1 is 16x4 matrix

    x_2 = x + 0.5*h*f1 # update x for 2nd cartpole() call
    print
    f2 = cartpole(x_2,u).flatten()
    d2f2 = dBdu_cartpole(x_2[0],x_2[1],x_2[2],x_2[3],u)

    x_3 = x + 0.5*h*f2
    f3 = cartpole(x_3,u).flatten()
    d2f3 = dBdu_cartpole(x_3[0],x_3[1],x_3[2],x_3[3],u)

    x_4 = x + 0.5*h*f3
    f4 = cartpole(x_4,u).flatten()
    d2f4 = dBdu_cartpole(x_4[0],x_4[1],x_4[2],x_4[3],u)
    d2ft = xdd_1 + (h/6.0)*(d2f1 + 2*d2f2 + 2*d2f3 + d2f4)    
    return d2ft 


ทดสอบโดยใช้ค่าสุ่ม รันเซลล์นี้หลายครั้ง ผลต่างจะต้องมีค่าน้อย

In [29]:
x_test = np.random.rand(4,)
u_test = np.random.rand()
Bx_ad_test = dBdx(x_test,u_test) # from JAX
Bx_md_test = dBdx_cartpole_rk4(x_test,u_test) # from symbolic
np.linalg.norm(Bx_md_test - Bx_ad_test)

np.float32(0.0025793996)

In [30]:
x_test = np.random.rand(4,)
u_test = np.random.rand()
Au_ad_test = dAdu(x_test,u_test) # from JAX
Au_md_test = dAdu_cartpole_rk4(x_test,u_test) # from symbolic
np.linalg.norm(Au_md_test - Au_ad_test)

np.float32(0.0022751808)

In [31]:
x_test = np.random.rand(4,)
u_test = np.random.rand()
Bu_ad_test = dBdu(x_test,u_test) # from JAX
Bu_md_test = dBdu_cartpole_rk4(x_test,u_test) # from symbolic
np.linalg.norm(Bu_md_test - Bu_ad_test)

np.float32(6.666676e-05)

**หมายเหตุ :** จากการทดสอบนี้เราตั้งข้อสังเกตว่าแต่ละสมาชิกของเมทริกซ์/เวกเตอร์ที่ได้จากพจน์เฮสเซียนมีค่าน้อยมาก 
ดังนั้นผลจากพจน์เหล่านี้ในการปรับค่าอาจไม่คุ้มค่าเมื่อเปรียบเทียบกับเวลาที่ใช้ในการคำนวณ กล่าวคือการใช้อัลอกริทึมเกาส์-นิวตันในวิธี iLQR 
น่าจะเป็นทางเลือกที่ดีกว่าอัลกอริทึมนิวตันแบบเต็มในวิธี DDP 

## บรรณานุกรม

1. R. Tedrake and the Drake Development Team. ["Drake: Model-based design and verification
 for robotics"](https://drake.mit.edu). 2019.

2. Z. Manchester et.al. [16-745 Optimal Control & Reinforcement Learning, 
Course materials](https://optimalcontrol.ri.cmu.edu/#learning-resources), Carnegie Mellon University. 2025.

3. R. Tedrake. [Underactuated Robotics: Algorithms for Walking, Running, Swimming, Flying, and Manipulation (Course Notes for MIT 6.832)](https://underactuated.csail.mit.edu). 2023. 


<div align="center">
<img src="https://raw.githubusercontent.com/dewdotninja/sharing-github/refs/heads/master/dewninja_logo50.jpg" alt="dewninja"/>
</div>
<div align="center">dew.ninja 2025</div>